# Transformers pour la classification de texte

L'objectif de ce TP est d'implémenter une version simplifiée d'un Transformer pour résoudre un problème de classification de texte.

Nous utiliserons comme exemple illustratif une base de données présente dans la librairie ```Keras``` consistant en des critiques de films postées sur le site IMDB, accompagnées d'une note qui a été binarisée pour révéler le caractère positif, ou négatif, de la critique.

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

## Implémentation d'un bloc de base de Transformer


<center><img src="https://drive.google.com/thumbnail?id=1leAVyTZJ2gZ26CFoauMtmu7T4Jo-Crc_&sz=w1000" width=200> </center>
<caption><center> Figure 1: Schéma de l'architecture de BERT</center></caption>

La figure ci-dessus présente l'architecture de BERT. Le bloc de base d'un Transformer est composé d'un bloc de *Self-Attention*, d'une couche de ```Layer Normalization```, d'une couche dense et enfin d'une nouvelle couche de ```Layer Normalization```. L'idée de la couche de ```Layer Normalization```est de faciliter le processus d'apprentissage en renormalisant les activations après chaque couche du réseau afin d'éviter que les valeurs ne s'écartent trop de 0.
Prêtez également bien attention aux **couches résiduelles**.

Pour implémenter la *Self-Attention*, vous pouvez utiliser la fonction ```Multi-Head Attention``` (à vous de regarder quels en sont les paramètres dans la documentation).

**Rappel**: Une couche d'Attention *Multi-Head*  se présente sous la forme ci-dessous à gauche, avec le mécanisme d'attention détaillé à droite :


<center>

<img src="https://drive.google.com/thumbnail?id=1UTozEHtsZ3xy61XJqn_Eug-7mn7bFp9m&sz=w1000">
<img src="https://drive.google.com/thumbnail?id=1aTttpp1OOasVVZAi3lWwosh68VnBjQnz&sz=w1000">
</center>

**D'après vous, combien de paramètres comporte une couche d'attention à 2 têtes, pour un *Embedding* de dimension 32 ?**


Pour implémenter un bloc Transformer, je vous propose dans ce TP d'adopter une notation ressemblant beaucoup à Pytorch. Nous allons instancier une classe *Layer*, et notamment les deux fonctions :    
-  ```__init__()```, un constructeur dans lequel nous initialisons toutes les couches que nous allons utiliser
-  ```call()``` appelé pour la prédiction et qui décrit comment les différentes couches sont appliquées aux entrées.

In [23]:
class TransformerBlock(keras.layers.Layer):
    # embed_dim désigne la dimension des embeddings maintenus à travers les différentes couches,
    # et num_heads le nombre de têtes de la couche d'attention.
    # DANS CETTE FONCTION, ON NE FAIT QUE DEFINIR LES COUCHES
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        # Définition des différentes couches qui composent le bloc
        # Couche d'attention
        self.att = keras.layers.MultiHeadAttention(num_heads, embed_dim)
        # Première couche de Layer Normalization
        self.layernorm1 = keras.layers.LayerNormalization()
        # Couche Dense (Feed-Forward)
        self.ffn = Dense(embed_dim, activation='relu')
        # Deuxième couche de normalisation
        self.layernorm2 = keras.layers.LayerNormalization()

    # DANS CETTE FONCTION, ON APPELLE EXPLICITEMENT LES COUCHES DEFINIES DANS __init__
    # ON PROPAGE DONC LES ENTREES inputs A TRAVERS LES DIFFERENTES COUCHES POUR OBTENIR
    # LA SORTIE
    def call(self, inputs):
        # Application des couches successives aux entrées
        out1 = self.att(inputs, inputs)
        out2 = self.layernorm1(out1 + inputs)
        out3 = self.ffn(out2)
        out4 = self.layernorm2(out3 + out2)
        return out4

## Implémentation de la double couche d'Embedding

La séquence d'entrée est convertie en *Embedding* de dimension ```embed_dim```.
L'*Embedding* final est constitué de la somme de deux *Embedding*, le premier encodant un mot, et le second encodant la position du mot dans la séquence.

La couche d'*Embedding* de Keras (```layers.Embedding```) est une sorte de table associant à un indice en entrée un vecteur de dimension ```embed_dim```. Chaque coefficient de cette table est en fait un paramètre apprenable.

**D'après vous combien de paramètres contiendrait une couche d'*Embedding* associant un vecteur de dimension 32 à chacun des 20000 mots les plus courants du vocabulaire extrait de la base de données que nous allons utiliser ?
Et combien pour l'*Embedding* qui associe un vecteur de dimension 32 à chaque position d'un séquence de longueur ```maxlen``` ?**

In [24]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        # Définition des différentes couches qui composent le bloc Embedding
        # Embedding de mot
        self.token_emb = keras.layers.Embedding(vocab_size, embed_dim)
        # Embedding de position
        self.pos_emb = keras.layers.Embedding(maxlen, embed_dim)

    def call(self, x):
        # Calcul de l'embedding à partir de l'entrée x
        # ATTENTION : UTILISER UNIQUEMENT DES FONCTIONS TF POUR CETTE PARTIE
        # Récupération de la longueur de la séquence
        maxlen = tf.shape(x)[-1]
        # Création d'un vecteur [0, 1, ..., maxlen] des positions associées aux
        # mots de la séquence (fonction tf.range)
        positions = tf.range(maxlen)
        # Calcul des embeddings de position
        positions_emb = self.pos_emb(x)
        # Calcul des embeddings de mot
        words_emb = self.token_emb(x)
        return positions_emb + words_emb

## Préparation de la base de données

In [25]:
# Taille du vocabulaire considéré (on ne conserve que les 20000 mots les plus courants)
vocab_size = 20000
# Taille maximale de la séquence considérée (on ne conserve que les 200 premiers mots de chaque commentaire)
maxlen = 200

# Chargement des données de la base IMDB
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)

print(len(x_train), "séquences d'apprentissage")
print(len(x_val), "séquences de validation")

# Padding des séquences : ajout de "0" pour compléter les séquences trop courtes
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

25000 séquences d'apprentissage
25000 séquences de validation


## Création du modèle

Pour assembler le modèle final, il faut, partant d'une séquence de longueur ```maxlen```, calculer les Embedding puis les fournir en entrée d'une série de blocs Transformer. Pour ce TP, **commencez par ne mettre qu'un seul bloc Transformer**. Vous pourrez en ajouter plus tard si vous le souhaitez.

Pour construire la tête de projection du réseau, vous pouvez moyenner les activations en sortie du bloc Transformer par élément de la séquence grâce à un *Global Average Pooling* (1D !), à relier à une couche dense (par exemple comportant 20 neurones) et enfin à la couche de sortie du réseau.

In [26]:
embed_dim = 32  # Dimension de l'embedding pour chaque mot
num_heads = 2  # Nombre de têtes d'attention

# A COMPLETER
inputs = keras.layers.Input(shape=(maxlen,))
embed_layer = TokenAndPositionEmbedding(vocab_size=vocab_size, maxlen=maxlen, embed_dim=embed_dim)(inputs)
transformer_layer = TransformerBlock(embed_dim=embed_dim, num_heads=num_heads)(embed_layer)
globalAvgPooling = keras.layers.GlobalAveragePooling1D()(transformer_layer)
linear = Dense(20, activation='relu')(globalAvgPooling)
outputs = Dense(1, activation='softmax')(linear)


model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding_6  │ (None, 200, 32)        │       646,400 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_6             │ (None, 200, 32)        │         9,600 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_3      │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 20)             │           660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 656,681 (2.51 MB)

 Trainable params: 656,681 (2.51 MB)

 Non-trainable params: 0 (0.00 B)

Enfin vous pouvez lancer l'apprentissage, avec par exemple l'optimiseur Adam. Inutile de lancer de trop nombreuses *epochs*, le réseau sur-apprend très vite !

In [27]:
# A COMPLETER
model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)
with tf.device('/GPU:0'):
    history = model.fit(
        x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val)
    )

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.5005 - loss: 0.4963 - val_accuracy: 0.5000 - val_loss: 0.2968
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4935 - loss: 0.1842 - val_accuracy: 0.5000 - val_loss: 0.3159
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5028 - loss: 0.1030 - val_accuracy: 0.5000 - val_loss: 0.3877
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5001 - loss: 0.0652 - val_accuracy: 0.5000 - val_loss: 0.4898
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5027 - loss: 0.0379 - val_accuracy: 0.5000 - val_loss: 0.6322


# Utilisation d'un modèle pré-entraîné

L'implémentation d'un Transformer *from scratch* comme dans la section précédente n'est en fait pas réellement recommandée. Il est souvent bien plus pertinent d'utiliser un modèle pré-entraîné et de faire du transfert d'apprentissage.

Dans cette 2nde partie, nous allons utiliser des modèles fournis par HuggingFace pour appréhender les différentes parties d'un système de classification de texte, puis pour résoudre le problème de la partie précédente d'une manière plus satisfaisante.

## Tokenization

La tokenization désigne l'opération de découpage du texte initial en une séquence de *tokens*, c'est-à-dire d'unités indivisibles de texte. Dans la partie précédente, nous avons implicitement utilisé une simple Tokenization au niveau mot.

Pour jouer un peu avec ce concept, chargeons le Tokenizer du modèle CAMEMBERT. CAMEMBERT est une version de BERT entraînée sur des données uniquement en français. Le Tokenizer a donc été préparé uniquement pour des mots en français.




In [ ]:
from transformers import CamembertTokenizer

# Charger le tokenizer CamemBERT
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")

# Phrase en français
sentence = "J'apprécie les fruits au sirop."

# Tokenisation de la phrase
encoded = tokenizer(sentence, return_tensors="tf")

# Afficher les IDs des tokens
token_ids = encoded["input_ids"].numpy()[0]  # Convertir en numpy pour affichage
print("IDs des tokens :", token_ids)

# Associer chaque ID au mot correspondant
tokens = [tokenizer.decode([token_id]) for token_id in token_ids]

# Afficher les tokens avec leur ID
print("\nCorrespondance ID -> Token :")
for token_id, token in zip(token_ids, tokens):
    print(f"{token_id} -> {token}")


Sur cet exemple, on a l'impression que la Tokenization de CAMEMBERT a été également réalisée au niveau mot. On repère les tokens spéciaux \<s\> (pour démarrer une phrase) et \</s\> (pour la terminer). *Notez ici pour la suite que cette phrase compte 10 tokens.*

En réalité, le Tokenizer est hybride : il décompose les phrases en mots si ceux-ci sont très communs, mais aussi en syllabes ou même en simples caractères.

**Travail à faire : Essayez de trouver des exemples de phrases pour faire apparaître des tokens qui ne sont pas des mots.**

## Complétion de texte masqué par CAMEMBERT

Les modèles de type BERT, comme CAMEMBERT, sont pré-entraînés sur de vastes corpus de textes de manière non supervisée. La méthode d'entraînement la plus commune est de leur faire compléter des morceaux de phrase qui ont été masqués à l'aide d'un token spécial.


Il est intéressant de tester la capacité du modèle à compléter un texte masqué en appliquant la procédure suivante :

In [ ]:
from transformers import TFCamembertForMaskedLM
import tensorflow as tf

# Chargement du modèle Camembert
model = TFCamembertForMaskedLM.from_pretrained("camembert-base")

# Phrase en français masquée
sentence = "J'apprécie les <mask> au sirop."

# Tokenisation de la phrase
encoded = tokenizer(sentence, return_tensors="tf")

# Prédiction du modèle sur la phrase
outputs = model(encoded)

# Affichage de la dimension du tenseur prédit
print(outputs.logits.shape)


Le tenseur prédit par le modèle CAMEMBERT sur la phrase d'exemple est de taille $1 \times 10 \times 32005$ :    
- le 1 correspond à la taille du batch (ici simplement 1)
- le 10 correspond au nombre de tokens de la séquence, c'est la valeur que nous avions trouvée précédemment. Le symbole \<mask\> compte pour un token.
- La veleur 32005 correspond au nombre de tokens de la tokenization de CAMEMBERT. On obtient ainsi une distribution de probabilité sur tous ces tokens.

Pour prédire le mot masqué, il suffit de retrouver parmi les 10 tokens, l'indice du token \<mask\> et de regarder la distribution de probabilité prédite par le modèle. On peut ainsi afficher les tokens ayant la probabilité la plus élevée de compléter le trou dans la phrase.

In [ ]:
### A COMPLETER
# Récupérer l'index du token masqué
mask_index = tf.where(encoded["input_ids"] == tokenizer.mask_token_id).numpy().flatten()[1]

# Obtenir les logits des prédictions pour le token masqué
logits = ...

# Il faut appliquer la fonction softmax pour obtenir une distribution de probabilité
probs = ...

# On collecte les 5 prédictions les plus probables, ainsi que leur probabilité
top_5 = tf.math.top_k(probs, k=5)

# Associer chaque token ID à son mot et sa probabilité
predictions = [(tokenizer.decode([idx]).strip(), prob) for idx, prob in zip(top_5.indices.numpy(), top_5.values.numpy())]

# Afficher les mots prédits avec leurs probabilités
print("Mots prédits avec probabilités :")
for word, prob in predictions:
    print(f"{word}: {prob:.4f}")

## Transfert d'apprentissage d'un modèle BERT pré-entraîné

Revenons maintenant au problème initial de classification de reviews IMDB.

Nous devons d'abord résoudre un petit problème : la base de données fournie par Keras ne contient que les identifiants des mots écrits dans les reviews, afin de s'abstraire des problématiques de tokenization.

Pour pouvoir utiliser le Tokenizer du modèle que nous allons choisir, il nous faut disposer des phrases originelles. Nous allons adopter une méthode un peu imparfaite et nous allons reconstruire les reviews en utilisant le dictionnaire (index vers mot) fourni par Keras. Malheureusement, comme nous avons choisi de ne conserver que les 20 000 mots les plus courants, il manquera certains mots !

In [ ]:
# Récupération du dictionnaire mot->index
word_index = keras.datasets.imdb.get_word_index()

# Insertion des tokens rajoutés par Keras
word_index = {word: (index + 3) for word, index in word_index.items()}
word_index["<PAD>"] = 0   # token de padding
word_index["<START>"] = 1  # token de début de review
word_index["<UNK>"] = 2    # Token pour les mots inconnus
word_index["<UNUSED>"] = 3
print(word_index)

# Inversion du dictionnaire : index->mot
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

print(reverse_word_index)

Nous pouvons maintenant, à l'aide du dictionnaire, retrouver les phrases initiales de la base de données :



In [ ]:
def indices_to_text(indices):
    return " ".join(reverse_word_index.get(i, "<UNK>") for i in indices)

# Affichage de la première review de la base d'entraînement
decoded_review = indices_to_text(x_train[0])

print(decoded_review)

On observe en effet qu'il manque des mots !

Si nous chargeons maintenant le Tokenizer du modèle BERT, nous pouvons ainsi observer la version tokenizée de la première review de la base d'apprentissage. Vous comprenez donc que le processus que nous venons d'appliquer nous a permis de retrouver la base initiale depuis la Tokenization de Keras, afin de pouvoir préparer cette base à la classification par BERT en lui appliquant le Tokenizer correspondant !

In [ ]:
from transformers import AutoTokenizer

# Chargement du Tokenizer du modèle ALBERT
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

# Tokenization de la première review du dataset
tokenizer(decoded_review, return_tensors="np", padding=True)

Maintenant que cette preuve de concept fonctionne nous pouvons appliquer le même processus à l'ensemble de la base d'apprentissage et de la base de validation :

In [ ]:
# On convertit maintenant l'ensemble des reviews en texte
texts_train = [indices_to_text(review) for review in x_train]
texts_val = [indices_to_text(review) for review in x_val]

# On applique le Tokenizer du modèle ALBERT sur toutes les reviews
x_train_bert = tokenizer(texts_train, return_tensors="np", padding=True)
x_val_bert = tokenizer(texts_val, return_tensors="np", padding=True)

# Enfin, on prépare les tenseurs pour l'apprentissage en les passant au bon format tensorflow
x_train_bert_tensors = {key: tf.convert_to_tensor(value) for key, value in x_train_bert.items()}
x_val_bert_tensors = {key: tf.convert_to_tensor(value) for key, value in x_val_bert.items()}

Nous pouvons maintenant lancer le transfert d'apprentissage du modèle ALBERT, qui est une version plus compacte de BERT. Attention cette dernière cellule est longue ! Cette partie est plus pour vous donner un exemple de fine-tuning mais de nombreux tutorials existent pour l'accélérer (par exemple, en utilisant des modèles quantifiés)

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from transformers import AdamWeightDecay

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
model.summary()
# Lower learning rates are often better for fine-tuning transformers
optimizer = AdamWeightDecay(learning_rate=3e-5)
model.compile(optimizer=optimizer)  # No loss argument!

model.fit(x_train_bert_tensors, y_train, batch_size=8, validation_data=(x_val_bert_tensors, y_val))